In [ ]:
import os
import pandas as pd
import json

from sklearn.impute import SimpleImputer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Customers Table

This notebook contains exploratory data analysis (EDA) and preprocessing steps for the `customers` table.

Key preprocessing tasks include:
- Formatting and imputing the `gender` column using data generated by a ChatGPT model in `api_data_download.ipynb`
- Importing exact customer addresses from the table produced in `api_data_download.ipynb`, enriched via the Google Maps API
- Cleaning and imputing the `city`, `birth_date`, `education`, and `occupation` columns
- Calculating `age` based on the `birth_date` and imputing missing values where necessary


## Load customers data

In [4]:
file_path = 'data/customers.csv'
customers_df = pd.read_csv(file_path)
customers_df.head()

,id,name,street_address,city,state,zipcode,birth_date,gender,education,occupation
0,df83ec2d0d409395c0d8c2690cfa8b67,Cynthia Barnfield,44 NE Meikle Pl,Portland,Oregon,97213.0,2009-09-10,female,High School,NaN
1,6aec7ab2ea0d67161dac39e5dcabd857,Elizabeth Smith,7511 SE Harrison St,Portland,Oregon,97215.0,1956-12-15,female,College,Blue Collar
2,0c54340672f510fdb9d2f30595c1ab53,Richard Pabla,1404 SE Pine St,Portland,Oregon,97214.0,1960-12-18,male,College,Education & Health
3,f0d9ce833ddc1f73c1e0b55bdebf012e,Charles Baker,12271 N Westshore Dr,Portland,Oregon,97217.0,2105-07-19,male,Graduate Degree,SALES
4,3720379163f6b46944db6c98c0485bfd,Ronald Lydon,5321 NE Skyport Way,NaN,Oregon,97218.0,1961-03-14,male,Graduate Degree,Blue Collar


In [ ]:
# Table contains 2000 unique customers
customers_df.shape

(2000, 10)

In [ ]:
# Check number of customers in the checkouts table
# Conclusion - checkouts table has 2000 rows and 2000 unique customers
# In checkouts table we have one record per unique customer
file_path = 'data/checkouts.csv'
checkouts_df = pd.read_csv(file_path)
checkouts_df['patron_id'].nunique()

2000

In [ ]:
# Check missing values
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              2000 non-null   object
 1   name            2000 non-null   object
 2   street_address  2000 non-null   object
 3   city            1906 non-null   object
 4   state           1898 non-null   object
 5   zipcode         1999 non-null   object
 6   birth_date      1893 non-null   object
 7   gender          1899 non-null   object
 8   education       1896 non-null   object
 9   occupation      1893 non-null   object
dtypes: object(10)
memory usage: 156.4+ KB


## Set id as index

In [103]:
customers_df = customers_df.set_index(keys=['id'], drop=True)

## Fill gender missing values

In [ ]:
# Missing gender values are determined based on customer name by prompting open ai gpt-4 model
# Notebook : api_data_download.ipynb
# Section title: "Get gender for missing entries in customer table"

map_dict = {}

with open('new_data/gender_map.json') as f:
    map_dict = json.load(f)
    
for customer_id in map_dict:
    customers_df.loc[customer_id, 'gender'] = map_dict[customer_id]
    
customers_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, df83ec2d0d409395c0d8c2690cfa8b67 to 2d1d65505400e568a045c828c9ebe365
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            2000 non-null   object
 1   street_address  2000 non-null   object
 2   city            1906 non-null   object
 3   state           1898 non-null   object
 4   zipcode         1999 non-null   object
 5   birth_date      1893 non-null   object
 6   gender          2000 non-null   object
 7   education       1896 non-null   object
 8   occupation      1893 non-null   object
dtypes: object(9)
memory usage: 220.8+ KB


## Get full adress data

In [ ]:
# Customer full adress was obtained by using python google maps library
# Notebook : api_data_download.ipynb
# Section title: "Get location data for customers"

adress_map_df = pd.read_csv('new_data/customer_location_map.csv', index_col='id')
adress_map_df.head()

,full_adress,latitude,longitude
id,,,
df83ec2d0d409395c0d8c2690cfa8b67,"44 NE Meikle Pl, Portland, OR 97213, USA",45.523503,-122.618769
6aec7ab2ea0d67161dac39e5dcabd857,"7511 SE Harrison St, Portland, OR 97215, USA",45.509114,-122.586047
0c54340672f510fdb9d2f30595c1ab53,"1404 SE Pine St, Portland, OR 97214, USA",45.520618,-122.651398
f0d9ce833ddc1f73c1e0b55bdebf012e,"12271 N Westshore Dr, Portland, OR 97217, USA",45.613633,-122.693028
3720379163f6b46944db6c98c0485bfd,"5321 NE Skyport Way, Portland, OR 97218, USA",45.576464,-122.608994


In [ ]:
# Merge full adress with customer data
customers_df = pd.concat([customers_df,adress_map_df], axis=1)

## Clean zipcode

In [ ]:
# Format and clean zipcode data
customers_df['zipcode'] = customers_df['zipcode'].str.extract(r'(\d+(?:\.\d+)?)')

## Clean education and ocupation

In [ ]:
# Format and clear occupation and education
customers_df['education'] = (
    customers_df['education']
    .str.lower()                    # lowercase
    .str.strip()                   # remove leading/trailing spaces
    .str.replace(r'\s+', ' ', regex=True)  # collapse multiple spaces
)

customers_df['occupation'] = (
    customers_df['occupation']
    .str.lower()                    # lowercase
    .str.strip()                   # remove leading/trailing spaces
    .str.replace(r'\s+', ' ', regex=True)  # collapse multiple spaces
)

In [ ]:
# Confirm value counts by unique education values
customers_df['education'].value_counts()

education
high school        506
others             476
college            461
graduate degree    453
Name: count, dtype: int64

In [ ]:
# Confirm value counts by unique occupation values
customers_df['occupation'].value_counts()

occupation
admin & support       301
education & health    275
others                269
blue collar           265
tech                  263
sales                 261
business & finance    259
Name: count, dtype: int64

In [ ]:
# Check relationships between ocupation and education
edu_vs_ocu_df = customers_df[['education', 'occupation']].value_counts().unstack(fill_value=0)
edu_vs_ocu_df = edu_vs_ocu_df/edu_vs_ocu_df.sum(axis=0)
edu_vs_ocu_df

occupation,admin & support,blue collar,business & finance,education & health,others,sales,tech
education,,,,,,,
college,0.250000,0.248980,0.237903,0.226562,0.243243,0.231076,0.258964
graduate degree,0.260563,0.208163,0.205645,0.277344,0.250965,0.235060,0.235060
high school,0.264085,0.285714,0.274194,0.246094,0.277992,0.243028,0.270916
others,0.225352,0.257143,0.282258,0.250000,0.227799,0.290837,0.235060


In [ ]:
# Check relationships between ocupation and education
edu_vs_ocu_df = customers_df[['occupation', 'education']].value_counts().unstack(fill_value=0)
edu_vs_ocu_df = edu_vs_ocu_df/edu_vs_ocu_df.sum(axis=0)
edu_vs_ocu_df

education,college,graduate degree,high school,others
occupation,,,,
admin & support,0.163218,0.172093,0.157233,0.141593
blue collar,0.140230,0.118605,0.146751,0.139381
business & finance,0.135632,0.118605,0.142558,0.154867
education & health,0.133333,0.165116,0.132075,0.141593
others,0.144828,0.151163,0.150943,0.130531
sales,0.133333,0.137209,0.127883,0.161504
tech,0.149425,0.137209,0.142558,0.130531


## Analyze age

In [ ]:
# Calculate customer age based on birth date (compared to 2019 year)
customers_df['birth_date'] = pd.to_datetime(customers_df["birth_date"], errors="coerce")
reference_date = pd.Timestamp("2019-01-01")
customers_df['age'] = (reference_date - customers_df['birth_date']).dt.days/356

def check_age_invalid(row: pd.Series):
    """
    Checks if customer age is valid or not.

    Args:
        row (pd.Series)): Single data frame row

    Returns:
        True if age is valid, False if it is not
    """
    
    # Define minimum age for a given education in USA
    min_age_by_education = {
        'high school': 17,
        'others': 16, 
        'college': 22,  
        'graduate degree': 24  
    }
    # Min working age
    min_working_age = 14
    
    age = row['age']
    occupation = row['occupation']
    education = row['education']

    # Check age is not realistic
    if age < 0 or age > 100:
        return True
    # Check if age is valid in respect to education level
    elif not pd.isna(education):
        if age <= min_age_by_education[education]:
            return True
        else:
            return False
    # For customers with occupation, check if age is above minimum allowed age
    elif not pd.isna(occupation):
        if age <= min_working_age:
            return True
        else:
            return False
    else:
        return False
        
# Set age impute to True for each row where age is invalid
age_impute = customers_df.apply(check_age_invalid, axis=1)
customers_df.loc[age_impute, 'age'] = None

# Imput invalid age values with median
imp_median = SimpleImputer(strategy='median') 
customers_df[['age']] = imp_median.fit_transform(customers_df[['age']])
customers_df['age_imputed'] = age_impute

## Impute education based on job and vice versa

In [ ]:
# Impute missing 'occupation' with the most frequent occupation for each customer education level
# Impute missing 'education' with the most frequent education for each customer occupation

customers_df['impute_occupation'] = pd.isna(customers_df['occupation'])
customers_df['impute_education'] = pd.isna(customers_df['education'])

customers_df['occupation'] = customers_df['occupation'].fillna(
    customers_df.groupby('education')['occupation'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA)
)

customers_df['education'] = customers_df['education'].fillna(
    customers_df.groupby('occupation')['education'].transform(lambda x: x.mode()[0] if not x.mode().empty else pd.NA)
)

# If both, ocupation and education values are missing, then impute with most frequent value
imputer = SimpleImputer(strategy='most_frequent')
customers_df[['occupation','education']] = imputer.fit_transform(customers_df[['occupation','education']])

In [ ]:
# Save processed data
customers_df.to_csv('data_preprocessed/customers_processed.csv')